In [11]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from genaicore import azure_openai_text_embeddings_llm
from langchain_experimental.text_splitter import SemanticChunker
# Generate embeddings
embeddings = azure_openai_text_embeddings_llm
# Load the report (Ensure the file is in the working directory)
pdf_name = "Att 5. Air Quality Assessment 4567 Old Northern Road Maroota.pdf"
loader = PyMuPDFLoader(pdf_name)
pages = loader.load()

# Split into manageable chunks
# text_splitter = CharacterTextSplitter(
#     separator="\n", chunk_size=1000, chunk_overlap=150, length_function=len
# )
text_splitter = SemanticChunker(embeddings)
docs = text_splitter.split_documents(pages)

# Remove unwanted whitespaces
for doc in docs:
    doc.page_content = doc.page_content.replace("\n", "")


In [12]:
from langchain_community.vectorstores import FAISS



# Create a FAISS vector database
db = FAISS.from_documents(docs, embeddings)

# Build a retriever
# retriever = db.as_retriever(search_kwargs={"k": 5})  # Retrieve top 5 relevant chunks
retriever = db.as_retriever(search_kwargs={"k": 10, "semantic_search": True})


In [13]:
# Load environmental standards
standards_file = "Environment_standards.txt"

with open(standards_file, "r") as f:
    standards = f.read()


In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from genaicore import azure_gpt4_openai_text_chat_llm
# Define LLM
llm = azure_gpt4_openai_text_chat_llm

# 1️⃣ Code: Checking Exceedances in Operational Phase Impact Assessment

In [47]:
# Query to check exceedances of impact assessment criteria (PCB Standards)
# query_exceedances = """
# Extract the following information from the attached PDF report:

# - All environmental parameters such as TSP, SO2, NO2, O3, Pb, PM2.5, PM10, deposited dust, CO, HF...etc
# - The values for each parameter.
# - The dates when these values were measured.
# - Any relevant notes related to these measurements.
# - Check if there are any cumulative values where applicable.

# Additionally, extract the values for each residential location (R1 to R20) and check if any of the values mentioned under these locations exceed the permissible limits for the following parameters:

# - PM10: 50 (24-hour limit), 25 (Annual limit)
# - PM2.5: 25 (24-hour limit), 8 (Annual limit)
# - TSP: 90 (Annual limit)

# The units of measurement (UOM) for these parameters are as follows:

# - Dust Deposited: g/m²/day
# - Others: µg/m³

# Please provide a comprehensive summary of your findings, including whether any values exceed the specified limits.
# """
query_exceedances="""
Extract the following information from the attached PDF report:

- All environmental parameters such as TSP, SO2, NO2, O3, Pb, PM2.5, PM10, deposited dust, CO, HF...etc
- The values for each parameter.
- The dates when these values were measured.
- Any relevant notes related to these measurements.
- Check if there are any cumulative values where applicable.
"""

# Retrieve relevant documents
retrieved_docs_exceedances = retriever.invoke(query_exceedances)
retrieved_text_exceedances = "\n\n".join([doc.page_content for doc in retrieved_docs_exceedances])

# Define structured prompt
# template_exceedances = """
# You are an AI environmental analyst. Analyze the retrieved data and determine if any environmental 
# parameters exceed the permissible PCB Standards.

# ### **Context: Retrieved Environmental Data**
# {context}

# ### **Environmental Standards**
# {standards}

# ### **Analysis Task**
# 1. Identify and compare environmental parameters with PCB standard limits.
# 2. List all parameters where exceedances were observed.
# 3. Format the findings in a structured table.

# ### **Output Format**
# | Parameter | Retrieved Value | Standard Limit | Pass/Fail |
# |-----------|----------------|---------------|-----------|
# | PM10      | 55 µg/m³       | 50 µg/m³      | ❌ Fail   |
# | PM2.5     | 20 µg/m³       | 25 µg/m³      | ✅ Pass   |

# ### **Conclusion**
# Summarize the number of **pass** vs. **fail** parameters.
# """

template_exceedances = '''You are an AI environmental analyst. Your task is to analyze the environmental report provided by a client and compare it with the given standards. and The response should be written in **Markdown** format.

### **Client Name:**  
Extract the name of the company from the given context and display it in the report. If the company name is missing, explicitly mention: **"Client name not found in the provided data."**  

---

## **🔹 Context: Retrieved Environmental Data**  
{context}

## **🔹 Environmental Standards**  
{standards}

---

## **🔹 Analysis Task**
1. **Identify and compare environmental parameters** (e.g., TSP, PM10, PM2.5, NO2, etc.) from the report with the corresponding standard limits.
2. **Ensure unit consistency** – if the client’s values are in different units, convert them to match the standards before comparison.
3. **Clearly state whether each parameter meets the standard (Pass/Fail).**
4. **Present the comparison in a structured table format.**
5. **If any data is missing in the provided context, explicitly mention it instead of making assumptions.**

---

## **🔹Example Output Format(these are example values but you want to get thse values from the {standards}**
### **Environmental Compliance Report of [Mention Client Name here]**

| Parameter | Client Value (Converted) | Standard Limit | Pass/Fail |
|-----------|--------------------------|----------------|-----------|
| |                                                         ✅ Pass 

---

### **🔹 Additional Notes**
- If a value is missing, mention **"Data Missing"** in the table instead of assuming.
- The report strictly follows the provided **context** and **standards**.
- Any necessary unit conversions are applied before comparison.

---

### **🔹 Conclusion**
Summarize the key findings:  
- Number of **parameters that passed**.  
- Number of **parameters that failed**.  
- If any **critical violations** exist.  

If any information is missing, list the **specific missing parameters** at the end.

---
'''



# Create prompt
prompt_exceedances = ChatPromptTemplate.from_template(template_exceedances)

# Construct the chain manually
formatted_prompt_exceedances = prompt_exceedances.invoke(
    {"context": retrieved_text_exceedances, "standards": standards}
)
response_exceedances = llm.invoke(formatted_prompt_exceedances)

# Parse the response
parsed_response_exceedances = StrOutputParser().invoke(response_exceedances)

# Print result
print(parsed_response_exceedances)


# Environmental Compliance Report

### **Client Name:**
**Client name not found in the provided data.**

---

## **🔹 Context: Retrieved Environmental Data**

### Extracted Environmental Parameters From Client Data:
- PM10
- PM2.5
- TSP
- Dust Deposition

### **🔹 Standarized Environmental Parameters and Their Limits**  
**Extracted Parameters and Permissible Limits:**

| Pollutant       | Averaging Period | Impact Assessment Criteria |
|-----------------|------------------|----------------------------|
| TSP             | Annual           | 90 µg/m³                   |
| PM10            | 24-Hour          | 50 µg/m³                   |
| PM10            | Annual           | 25 µg/m³                   |
| PM2.5           | 24-Hour          | 25 µg/m³                   |
| PM2.5           | Annual           | 8 µg/m³                    |
| Dust Deposition | Annual           | 2 g/m²/month               |

---

### **🔹 Analysis of Client’s Environmental Data Against Standards**

#### **Pol

In [1]:
import os
import subprocess
# Save markdown content to file
def save_markdown_to_file(documentation: str, file_path: str):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(documentation)

    print(f"Markdown content has been saved to {file_path}")

# Convert Markdown file to .docx using pandoc
def convert_md_to_docx(md_file: str, docx_file: str, template_file: str = None):
    try:
        command = ["pandoc", md_file, "-o", docx_file]
        if template_file:
            command.extend(["--reference-doc", template_file])
        
        subprocess.run(command, check=True)
        print(f"Successfully converted {md_file} to {docx_file}")
    except subprocess.CalledProcessError as e:
        print(f"Error during conversion: {e}")

In [4]:
markdown_file_path =  'Ensuring Appropriate Criteria at Sensitive Receptors.docx'
# save_markdown_to_file(parsed_response_exceedances, markdown_file_path)
docx_file_path = "Assessment-1.html"
# pdf_file_path = "video_documentation.pdf"

# Convert .md to .docx
convert_md_to_docx(markdown_file_path, docx_file_path)

Successfully converted Ensuring Appropriate Criteria at Sensitive Receptors.docx to Assessment-1.html


# 2️⃣ Code: Verifying Background Dust Concentrations

In [33]:
# Query for background dust concentrations in impact assessment
query_background_dust = """
Retrieve information on how background dust concentrations were considered in the operational 
phase impact assessment. Determine if they were analyzed according to  Standards.
"""

# Retrieve relevant documents
retrieved_docs_background = retriever.invoke(query_background_dust)
retrieved_text_background = "\n\n".join([doc.page_content for doc in retrieved_docs_background])

# Define structured prompt
template_background_dust = """
You are an AI environmental analyst. Analyze the retrieved environmental data to verify whether 
background dust concentrations were considered in relation to  Standards. The response should be written in **Markdown** format.

### **Context: Retrieved Environmental Data**
{context}

### **Environmental Standards**
{standards}

### **Analysis Task**
1. Identify whether background Dust Deposition were included in the impact assessment.
2. Compare the methodology used against  standards.
3. Summarize findings clearly.

### **Output Format**
#### **Background Dust Deposition Analysis**
- Background dust data analysis: **[Mention if available]**
- Consideration in impact assessment: **[Yes/No]**
- Alignment with  Standards: **[Yes/No]**
- Any missing data: **[List missing elements]**

### **Conclusion**
Summarize whether background concentrations were properly considered.
"""

# Create prompt
prompt_background_dust = ChatPromptTemplate.from_template(template_background_dust)

# Construct the chain manually
formatted_prompt_background_dust = prompt_background_dust.invoke(
    {"context": retrieved_text_background, "standards": standards}
)
response_background_dust = llm.invoke(formatted_prompt_background_dust)

# Parse the response
parsed_response_background_dust = StrOutputParser().invoke(response_background_dust)

# Print result
print(parsed_response_background_dust)


### **Background Dust Deposition Analysis**

- **Background dust data analysis:** Yes, the retrieved environmental data includes background dust data analysis.
  - *PM10:* Annual average concentration of 13 µg/m³.
  - *PM2.5:* Annual average concentration of 5.0 µg/m³.
  - *TSP:* Annual average concentration of 32.5 µg/m³.
  - *Dust Deposition:* Annual average deposition of 2.1 g/m²/month.
  
- **Consideration in impact assessment:** Yes, background dust levels were considered in the impact assessment. The report mentions that:
  - It is necessary to include background dust concentration and deposition levels to fully assess impacts against relevant air quality standards and goals.
  - Impact assessment criteria are applied to cumulative impacts due to the proposal and other sources, which implies the inclusion of existing background levels.
  - Even the one predicted exceedance of the 24-hour PM10 criterion at a receptor (R8) was based on cumulative background concentrations and not j

In [34]:
markdown_file_path =  'Verifying Background Dust Concentrations.md'
save_markdown_to_file(parsed_response_background_dust, markdown_file_path)
docx_file_path = "Verifying Background Dust Concentrations.docx"
# pdf_file_path = "video_documentation.pdf"

# Convert .md to .docx
convert_md_to_docx(markdown_file_path, docx_file_path)

Markdown content has been saved to Verifying Background Dust Concentrations.md
Successfully converted Verifying Background Dust Concentrations.md to Verifying Background Dust Concentrations.docx


# 3️⃣ Code: Ensuring Appropriate Criteria at Sensitive Receptors

In [ ]:
# Query for impact assessment criteria at sensitive receptors
# query_sensitive_receptors = """
# Retrieve information on whether the operational phase impact assessment considers the appropriate 
# impact assessment criteria at each sensitive receptor. List all receptors assessed.
# """
query_sensitive_receptors ='''Retrieve receptor locations along with all environmental parameters affecting them, 
including air pollutants (PM10, PM2.5, NO₂, SO₂, CO, O₃, VOCs), meteorological factors (wind speed, wind direction, temperature, humidity, pressure), 
emission sources (industrial emissions, vehicular emissions, construction dust, agricultural burning, power plants), 
dispersion factors (wind erosion, dry and wet deposition rates), and health impact indicators (AQI, toxicity, respiratory impact score).'''

# Retrieve relevant documents
retrieved_docs_receptors = retriever.invoke(query_sensitive_receptors)
retrieved_text_receptors = "\n\n".join([doc.page_content for doc in retrieved_docs_receptors])

# Define structured prompt
template_sensitive_receptors = """
You are an AI environmental analyst. Your task is to analyze the retrieved environmental data to determine whether 
the impact assessment used appropriate criteria at sensitive receptors. The response should be written in **Markdown** format.

Analyze the Retrieved receptor locations along with all environmental parameters affecting them, including:
- Air pollutants: PM10, PM2.5, NO₂, SO₂, CO, O₃, VOCs
- Meteorological factors: wind speed, wind direction, temperature, humidity, pressure
- Emission sources: industrial emissions, vehicular emissions, construction dust, agricultural burning, power plants
- Dispersion factors: wind erosion, dry and wet deposition rates
- Health impact indicators: AQI, toxicity, respiratory impact score

### **Context: Retrieved Environmental Data**
{context}

### **Environmental Standards**
{standards}

### **Analysis Task**
1. Identify whether appropriate assessment criteria were applied for each sensitive receptor.
2. List all receptors that were assessed.
3. Identify any gaps or missing assessments.

### **Example - Output Format**
#### **Sensitive Receptor Analysis**
| Sensitive Receptor | Criteria Applied | Meets Standards? |
|--------------------|------------------|-------------------|
| R1                 | PM10, PM2.5      | ✅ Yes            |
| R2                 | PM10 only        | ❌ No             |

### **Conclusion**
Summarize whether the proper impact assessment criteria were applied at each receptor.
"""

# template_sensitive_receptors = """
# You are an AI environmental analyst. Analyze the retrieved environmental data and compare the actual values 
# against the standards to assess if the impact assessment used appropriate criteria at sensitive receptors.

# ### **Context: Retrieved Environmental Data**
# {context}

# ### **Environmental Standards**
# {standards}

# ### **Analysis Task**
# 1. **Compare** the values of the environmental parameters (e.g., PM10, PM2.5, NO₂, etc.) at each sensitive receptor with the corresponding environmental standards.
# 2. **Assess** whether the appropriate assessment criteria were applied for each parameter.
# 3. **Identify** any discrepancies or gaps in the criteria applied for each sensitive receptor.
# 4. **Provide a detailed analysis** of the values against standards for each parameter.

# ### **Example - Output Format**
# #### **Sensitive Receptor Analysis**
# | Sensitive Receptor | Parameter        | Value    | Standard | Meets Standards? | Analysis       |
# |--------------------|------------------|----------|----------|------------------|----------------|
# | R1                 | PM10             | 5 µg/m³  | 10 µg/m³ | ✅ Yes           | The value is below the standard. Impact is within acceptable limits. |
# | R1                 | PM2.5            | 3 µg/m³  | 10 µg/m³ | ✅ Yes           | The value is well below the standard, no significant impact. |
# | R2                 | NO₂              | 18 ppb   | 40 ppb   | ✅ Yes           | The concentration of NO₂ is within the acceptable range. |
# | R2                 | SO₂              | 6 ppb    | 10 ppb   | ✅ Yes           | SO₂ concentration is low, no risk identified. |
# | R3                 | CO               | 0.9 ppm  | 1.0 ppm  | ✅ Yes           | CO concentration is within the safe limits. |
# | R3                 | O₃               | 35 ppb   | 40 ppb   | ✅ Yes           | Ozone levels are within the prescribed limit. |
# | R4                 | VOCs             | 24 µg/m³ | 20 µg/m³ | ❌ No            | VOC concentration exceeds the safe standard. Potential air quality concern. |

# ### **Conclusion**
# - For each receptor, the analysis compares the measured values of environmental parameters with the relevant standards.
# - Detailed insights highlight whether each parameter is within acceptable limits or poses potential risks.
# - Any discrepancies, such as exceeding standards, have been clearly identified, along with recommendations for mitigation or further assessment.
# """

# Create prompt
prompt_sensitive_receptors = ChatPromptTemplate.from_template(template_sensitive_receptors)

# Construct the chain manually
formatted_prompt_sensitive_receptors = prompt_sensitive_receptors.invoke(
    {"context": retrieved_text_receptors, "standards": standards}
)
response_sensitive_receptors = llm.invoke(formatted_prompt_sensitive_receptors)

# Parse the response
parsed_response_sensitive_receptors = StrOutputParser().invoke(response_sensitive_receptors)

# Print result
print(parsed_response_sensitive_receptors)


### **Environmental Impact Assessment Analysis**

The retrieved environmental data included detailed assessments of air pollutants, meteorological factors, emission sources, dispersion factors, and health impact indicators at various sensitive receptors around the project site. Here, we will analyze if the impact assessment utilized appropriate criteria for each sensitive receptor.

#### **List of Sensitive Receptors and Applied Criteria**
The receptors assessed in this analysis include:
1. Receptor 1 (R1)
2. Receptor 2 (R2)
3. Receptor 3 (R3)
4. Receptor 4 (R4)
5. Receptor 5 (R5)
6. Receptor 6 (R6)
7. Receptor 7 (R7)
8. Receptor 8 (R8)
9. Receptor 9 (R9)
10. Receptor 10 (R10)
11. Receptor 11 (R11)
12. Receptor 12 (R12)
13. Maroota Public School

#### **Environmental Parameters Assessed**
- **Air Pollutants**: PM10, PM2.5, TSP, dust deposition, NO₂, SO₂, CO, O₃, VOCs.
- **Meteorological Factors**: wind speed, wind direction, temperature, humidity, pressure.
- **Emission Sources**: indu

In [ ]:
markdown_file_path =  'Ensuring Appropriate Criteria at Sensitive Receptors.md'
save_markdown_to_file(parsed_response_sensitive_receptors, markdown_file_path)
docx_file_path = "Ensuring Appropriate Criteria at Sensitive Receptors.docx"
# pdf_file_path = "video_documentation.pdf"

# Convert .md to .docx
convert_md_to_docx(markdown_file_path, docx_file_path)

Markdown content has been saved to Ensuring Appropriate Criteria at Sensitive Receptors.md
Successfully converted Ensuring Appropriate Criteria at Sensitive Receptors.md to Ensuring Appropriate Criteria at Sensitive Receptors.docx


In [1]:
docx_file_path = "Ensuring Appropriate Criteria at Sensitive Receptors.docx"
pdf_file_path="Receptors.pdf"
# pdf_file_path = "video_documentation.pdf"

# Convert .md to .docx
convert_md_to_docx( docx_file_path,pdf_file_path)

NameError: name 'convert_md_to_docx' is not defined

In [9]:
from docx2pdf import convert



# Specify input .docx file and desired output .pdf file

input_file = "Exceedances in Operational Phase Impact Assessment.docx"

output_file = "Exceedances in Operational Phase Impact Assessment.pdf"



# Convert the document

convert(input_file, output_file) 



print("Conversion complete!") 


100%|██████████| 1/1 [00:17<00:00, 17.64s/it]

Conversion complete!


In [17]:
standards=""" 

PM10

PM2.5

TSP

24-hour

Annual

24-hour

Annual

Annual

50 µg/m³

25 µg/m³

25 µg/m³

8 µg/m³

90 µg/m³

"""

In [18]:
# Query for impact assessment criteria at sensitive receptors
# query_sensitive_receptors = """
# Retrieve information on whether the operational phase impact assessment considers the appropriate 
# impact assessment criteria at each sensitive receptor. List all receptors assessed.
# """
query_sensitive_receptors ='''Retrieve receptor locations along with all environmental parameters affecting them, 
including air pollutants (PM10, PM2.5, NO₂, SO₂, CO, O₃, VOCs), meteorological factors (wind speed, wind direction, temperature, humidity, pressure), 
emission sources (industrial emissions, vehicular emissions, construction dust, agricultural burning, power plants), 
dispersion factors (wind erosion, dry and wet deposition rates), and health impact indicators (AQI, toxicity, respiratory impact score).'''

# Retrieve relevant documents
retrieved_docs_receptors = retriever.invoke(query_sensitive_receptors)
retrieved_text_receptors = "\n\n".join([doc.page_content for doc in retrieved_docs_receptors])

# Define structured prompt
template_sensitive_receptors = """
1)      Report Heading : Name of the Company for which this assessment is done

2)      Remove italics from Environmental Parameters Assessed

3)      Remove the section List of Sensitive Receptors and Applied Criteria

4)      The values under column Criteria Applied Metrics is repetitive. Create a new title above the table under which have these values

5)      The response should be written in **Markdown** format.
 {standards}

PM10

PM2.5

TSP

24-hour

Annual

24-hour

Annual

Annual

50 µg/m³

25 µg/m³

25 µg/m³

8 µg/m³

90 µg/m³

 

5)      Let the table “Impact Assessment Criteria Applied“ contain following

 ### **Context: Retrieved Environmental Data**
{context}

Receptors

PM10

PM2.5

TSP

Meets

Observation

 

24-hour

Annual

24-hour

Annual

Annual

 

 

R1

Value

Value

Value

Value

Value

 

High cumulative PM2.5

R2

Value

Value

Value

Value

Value

 

Exceeds PM10 criteria cumulatively


"""

# template_sensitive_receptors = """
# You are an AI environmental analyst. Analyze the retrieved environmental data and compare the actual values 
# against the standards to assess if the impact assessment used appropriate criteria at sensitive receptors.

# ### **Context: Retrieved Environmental Data**
# {context}

# ### **Environmental Standards**
# {standards}

# ### **Analysis Task**
# 1. **Compare** the values of the environmental parameters (e.g., PM10, PM2.5, NO₂, etc.) at each sensitive receptor with the corresponding environmental standards.
# 2. **Assess** whether the appropriate assessment criteria were applied for each parameter.
# 3. **Identify** any discrepancies or gaps in the criteria applied for each sensitive receptor.
# 4. **Provide a detailed analysis** of the values against standards for each parameter.

# ### **Example - Output Format**
# #### **Sensitive Receptor Analysis**
# | Sensitive Receptor | Parameter        | Value    | Standard | Meets Standards? | Analysis       |
# |--------------------|------------------|----------|----------|------------------|----------------|
# | R1                 | PM10             | 5 µg/m³  | 10 µg/m³ | ✅ Yes           | The value is below the standard. Impact is within acceptable limits. |
# | R1                 | PM2.5            | 3 µg/m³  | 10 µg/m³ | ✅ Yes           | The value is well below the standard, no significant impact. |
# | R2                 | NO₂              | 18 ppb   | 40 ppb   | ✅ Yes           | The concentration of NO₂ is within the acceptable range. |
# | R2                 | SO₂              | 6 ppb    | 10 ppb   | ✅ Yes           | SO₂ concentration is low, no risk identified. |
# | R3                 | CO               | 0.9 ppm  | 1.0 ppm  | ✅ Yes           | CO concentration is within the safe limits. |
# | R3                 | O₃               | 35 ppb   | 40 ppb   | ✅ Yes           | Ozone levels are within the prescribed limit. |
# | R4                 | VOCs             | 24 µg/m³ | 20 µg/m³ | ❌ No            | VOC concentration exceeds the safe standard. Potential air quality concern. |

# ### **Conclusion**
# - For each receptor, the analysis compares the measured values of environmental parameters with the relevant standards.
# - Detailed insights highlight whether each parameter is within acceptable limits or poses potential risks.
# - Any discrepancies, such as exceeding standards, have been clearly identified, along with recommendations for mitigation or further assessment.
# """

# Create prompt
prompt_sensitive_receptors = ChatPromptTemplate.from_template(template_sensitive_receptors)

# Construct the chain manually
formatted_prompt_sensitive_receptors = prompt_sensitive_receptors.invoke(
    {"context": retrieved_text_receptors, "standards": standards}
)
response_sensitive_receptors = llm.invoke(formatted_prompt_sensitive_receptors)

# Parse the response
parsed_response_sensitive_receptors = StrOutputParser().invoke(response_sensitive_receptors)

# Print result
print(parsed_response_sensitive_receptors)


```markdown
# Air Quality Assessment Report

## Name of the Company for which this assessment is done

---

### Environmental Parameters Assessed

## Impact Assessment Criteria Applied

### Criteria Applied Metrics
- PM10 (24-hour): 50 µg/m³
- PM10 (Annual): 25 µg/m³
- PM2.5 (24-hour): 25 µg/m³
- PM2.5 (Annual): 8 µg/m³
- TSP (Annual): 90 µg/m³

### Impact Assessment Criteria Applied

| Receptors           | PM10 24-hour | PM10 Annual | PM2.5 24-hour | PM2.5 Annual | TSP Annual | Meets | Observation                        |
|---------------------|--------------|-------------|---------------|--------------|------------|-------|-------------------------------------|
| R1                  | Value        | Value       | Value         | Value        | Value      |       | High cumulative PM2.5               |
| R2                  | Value        | Value       | Value         | Value        | Value      |       | Exceeds PM10 criteria cumulatively  |
---

### **Context: Retrieved Environment

In [ ]:
template_concentrations = """
You are an AI environmental analyst. Your task is to analyze the retrieved environmental data and present it in a structured format. The response should be written in **Markdown** format.

### **Report Heading**
{company_name} - Environmental Impact Assessment

### **Environmental Parameters Assessed**
- Predicted Annual Average TSP Concentrations
- Predicted Annual Average PM10 Concentrations
- Predicted Annual Average PM2.5 Concentrations
- Predicted Annual Average Dust Deposition Levels
- Maximum predicted 24-Hour Average PM10 concentrations
- Maximum predicted 24-Hour Average PM2.5 concentrations

### **Criteria Applied Metrics**
| Parameter | 24-hour Standard | Annual Standard |
|-----------|------------------|-----------------|
| PM10      | 50 µg/m³         | 25 µg/m³        |
| PM2.5     | 25 µg/m³         | 8 µg/m³         |
| TSP       | 90 µg/m³         | -               |

### **Impact Assessment Criteria Applied**
| Receptors | PM10 (24-hour) | PM10 (Annual) | PM2.5 (24-hour) | PM2.5 (Annual) | TSP (Annual) | Meets Standards? | Observation |
|-----------|----------------|---------------|-----------------|----------------|--------------|------------------|-------------|
| R1        | Value          | Value         | Value           | Value          | Value        | ✅ Yes           | High cumulative PM2.5 |
| R2        | Value          | Value         | Value           | Value          | Value        | ❌ No            | Exceeds PM10 criteria cumulatively |

### **Conclusion**
Summarize the findings and highlight any exceedances or concerns.
"""

# Create prompt
prompt_concentrations = ChatPromptTemplate.from_template(template_concentrations)

# Construct the chain manually
formatted_prompt_concentrations = prompt_concentrations.invoke(
    {"context": retrieved_text_receptors, "company_name": "XYZ Company"}
)
response_concentrations = llm.invoke(formatted_prompt_concentrations)

# Parse the response
parsed_response_concentrations = StrOutputParser().invoke(response_concentrations)

# Print result
print(parsed_response_concentrations)